# Wine Quality Prediction Dashboard
This demo will consume a REST API fronting the latest trained model.

In [ ]:
import pandas as pd
import panel as pn
from utils.environment import init
import warnings
import numpy as np

warnings.filterwarnings("ignore")
np.random.seed(42)

experiment_id, client = init()

In [ ]:
# Define our sliders
fixed_acidity_float_slider = pn.widgets.EditableFloatSlider(
    name="Fixed Acidity(g/L)", start=0, end=15, step=0.1, value=7.0
)
volatile_acidity_float_slider = pn.widgets.EditableFloatSlider(
    name="Volatile Acidity (g/L)", start=0, end=2, step=0.1, value=1.0
)
citric_acid_float_slider = pn.widgets.EditableFloatSlider(name="Citric Acid (g/L)", start=0, end=2, step=0.1, value=1.0)
residual_sugar_float_slider = pn.widgets.EditableFloatSlider(
    name="Residual Sugar (g/L)", start=0, end=50, step=0.1, value=20.0
)
chlorides_float_slider = pn.widgets.EditableFloatSlider(name="Chlorides (g/L)", start=0, end=1, step=0.1, value=0.25)
free_sulfur_dioxide_float_slider = pn.widgets.EditableFloatSlider(
    name="Free Sulfur Dioxide (mg/L)", start=0, end=1000, step=0.1, value=100.0
)
total_sulfur_dioxide_float_slider = pn.widgets.EditableFloatSlider(
    name="Total Sulfur Dioxide (mg/L)", start=0, end=1000, step=0.1, value=200.0
)
density_float_slider = pn.widgets.EditableFloatSlider(name="Density", start=0, end=2, step=0.1, value=0.9)
pH_float_slider = pn.widgets.EditableFloatSlider(name="pH", start=2, end=5, step=0.1, value=2.0)
sulphates_float_slider = pn.widgets.EditableFloatSlider(name="Sulphates", start=0, end=2, step=0.1, value=0.5)
alcohol_float_slider = pn.widgets.EditableFloatSlider(name="Alcohol (%ABV)", start=0, end=20, step=0.1, value=10.0)

In [ ]:
def build_blank_result_row() -> pd.DataFrame:
    return pd.DataFrame(
        [
            {
                "fixed acidity": [],
                "volatile acidity": [],
                "citric acid": [],
                "residual sugar": [],
                "chlorides": [],
                "free sulfur dioxide": [],
                "total sulfur dioxide": [],
                "density": [],
                "pH": [],
                "sulphates": [],
                "alcohol": [],
                "prediction": [],
            }
        ]
    )

In [ ]:
results_df_pane = pn.widgets.DataFrame(build_blank_result_row(), width=1300)

In [ ]:
def clear_btn_action(event):
    results_df_pane.value = build_blank_result_row()

In [ ]:
from utils.rest import invoke_rest_endpoint
from ae5_tools import demand_env_var


def submit_btn_action(event):
    feature_data: list[dict] = [
        {
            "fixed acidity": fixed_acidity_float_slider.value,
            "volatile acidity": volatile_acidity_float_slider.value,
            "citric acid": citric_acid_float_slider.value,
            "residual sugar": residual_sugar_float_slider.value,
            "chlorides": chlorides_float_slider.value,
            "free sulfur dioxide": free_sulfur_dioxide_float_slider.value,
            "total sulfur dioxide": total_sulfur_dioxide_float_slider.value,
            "density": density_float_slider.value,
            "pH": pH_float_slider.value,
            "sulphates": sulphates_float_slider.value,
            "alcohol": alcohol_float_slider.value,
            "prediction": [],
        }
    ]
    json_payload: dict = {"dataframe_records": feature_data}
    predictions: dict = invoke_rest_endpoint(
        endpoint_url=demand_env_var(name="SELF_HOSTED_MODEL_ENDPOINT"), input_data=json_payload
    )
    predicted_quality: int = int(round(predictions["predictions"][0]))
    if predicted_quality < 3 or predicted_quality > 9:
        predicted_quality = 0
    feature_data[0]["prediction"] = predicted_quality

    row_df = pd.DataFrame(feature_data)

    if results_df_pane.value.loc[[0]]["prediction"][0] == []:
        results_df_pane.value = row_df
    else:
        results_df_pane.value = pd.concat([results_df_pane.value, row_df], ignore_index=True)

In [ ]:
# Define our buttons
submit_btn = pn.widgets.Button(name="Submit", button_type="primary")
submit_btn.on_click(submit_btn_action)

clear_btn = pn.widgets.Button(name="Clear", button_type="primary")
clear_btn.on_click(clear_btn_action)

In [ ]:
pn.extension()

title: str = "<h1>Wine Quality Dashboard</h1>"
description = pn.pane.HTML("""Feature selection and quality prediction for the Wine Quality model.""")

# Build layout
pn.Column(
    pn.Row(pn.Column(title, description)),
    pn.Row(
        pn.Column(
            fixed_acidity_float_slider,
            volatile_acidity_float_slider,
            citric_acid_float_slider,
            residual_sugar_float_slider,
            chlorides_float_slider,
            free_sulfur_dioxide_float_slider,
            total_sulfur_dioxide_float_slider,
            density_float_slider,
            pH_float_slider,
            sulphates_float_slider,
            alcohol_float_slider,
        ),
        pn.Column(pn.Row(submit_btn, clear_btn), results_df_pane),
    ),
    pn.Row(
        pn.Column(
            "### Feature Description",
            "*Fixed Acidity*: Acids in wine that do not readily evaporate.",
            "*Volatile Acidity*: The amount of acetic acid in wine, which at too high of levels can lead to an unpleasant, vinegar taste.",
            "*Citric Acid*: Found in small quantities, citric acid can add ‘freshness’ and flavor to wines.",
            "*Residual Sugar*: The amount of sugar remaining after fermentation stops.",
            "*Chlorides*: The amount of salt in the wine.",
            "*Free Sulfur Dioxide*: The free form of SO_2. It prevents microbial growth and the oxidation of wine.",
            "*Total Sulfur Dioxide*: The amount of free and bound forms of S02.",
            "*Density*: The density of water is close to that of water depending on the percent alcohol and sugar content.",
            "*pH*: Describes how acidic or basic a wine is on a scale from 0 (very acidic) to 14 (very basic); most wines are between 3-4 on the pH scale.",
            "*Sulphates*: A wine additive which can contribute to sulfur dioxide gas (S02) levels, wich acts as an antimicrobial and antioxidant.",
            "*Alcohol*: The percent alcohol content of the wine.",
        )
    ),
).servable("Wine Quality Dashboard")